# Предсказание задержек рейсов в аэропорту

## Подготовка датасета

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import json


Загружаем датасет и отбираем необходимые нам признаки:
* месяц
* день
* день недели
* авиалиния
* аэропорт отправления
* аэропорт назначения
* запланированное время отправления
* запланированное время прибытия
* задержка отправления (то, что нам необходимо предсказать)

In [2]:
dataset = pd.read_csv('dataset/flights.csv')
features = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY']
dataset = dataset[features]


C:\Users\papey08\AppData\Local\Temp/ipykernel_8288/1817673721.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('dataset/flights.csv')


In [3]:
dataset = dataset.sample(n=100000, random_state=2904)

dataset

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY
2249646,5,23,6,UA,EWR,SAN,902,-1.0
3339744,7,27,1,EV,ATL,PIA,2109,-7.0
3400532,7,31,5,WN,MDW,MCI,1230,0.0
3374019,7,29,3,UA,ORD,ALB,2050,1.0
5185513,11,20,5,OO,GEG,SLC,1635,36.0
...,...,...,...,...,...,...,...,...
2721992,6,21,7,B6,HPN,FLL,630,-8.0
4210448,9,19,6,AA,ORD,DCA,1922,-7.0
558299,2,6,5,DL,JFK,AUS,1735,-5.0
2540125,6,10,3,EV,BOS,RIC,905,-7.0


Получим список различных авиалиний и аэропортов и преобразуем категориальные переменные в числовые

In [4]:
dataset['AIRLINE_CODE'] = pd.factorize(dataset['AIRLINE'])[0]
dataset['ORIGIN_AIRPORT_CODE'] = pd.factorize(dataset['ORIGIN_AIRPORT'])[0]
dataset['DESTINATION_AIRPORT_CODE'] = pd.factorize(dataset['DESTINATION_AIRPORT'])[0]

In [5]:
dataset

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,AIRLINE_CODE,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE
2249646,5,23,6,UA,EWR,SAN,902,-1.0,0,0,0
3339744,7,27,1,EV,ATL,PIA,2109,-7.0,1,1,1
3400532,7,31,5,WN,MDW,MCI,1230,0.0,2,2,2
3374019,7,29,3,UA,ORD,ALB,2050,1.0,0,3,3
5185513,11,20,5,OO,GEG,SLC,1635,36.0,3,4,4
...,...,...,...,...,...,...,...,...,...,...,...
2721992,6,21,7,B6,HPN,FLL,630,-8.0,11,161,20
4210448,9,19,6,AA,ORD,DCA,1922,-7.0,5,3,78
558299,2,6,5,DL,JFK,AUS,1735,-5.0,4,51,14
2540125,6,10,3,EV,BOS,RIC,905,-7.0,1,30,95


Получим словари, в которых ключами будут числовые признаки, а значениями категориальные

In [6]:
airline_dict = dict(zip(dataset['AIRLINE'], dataset['AIRLINE_CODE']))
origin_airport_dict = dict(zip(dataset['ORIGIN_AIRPORT'], dataset['ORIGIN_AIRPORT_CODE']))
destination_airport_dict = dict(zip(dataset['DESTINATION_AIRPORT'], dataset['DESTINATION_AIRPORT_CODE']))


# Сохраним словари в json файлах чтобы использовать их в дальнейшем
with open('../json/airline_dict.json', 'w') as f:
    json.dump(airline_dict, f)

with open('../json/origin_airport_dict.json', 'w') as f:
    json.dump(origin_airport_dict, f)

with open('../json/destination_airport_dict.json', 'w') as f:
    json.dump(destination_airport_dict, f)


Удалим ненужные столбцы из датасета, а также записи, в которых присутствуют пустые значения

In [7]:
dataset = dataset.drop(columns=['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])
dataset = dataset.dropna()

In [8]:
dataset

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,AIRLINE_CODE,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE
2249646,5,23,6,902,-1.0,0,0,0
3339744,7,27,1,2109,-7.0,1,1,1
3400532,7,31,5,1230,0.0,2,2,2
3374019,7,29,3,2050,1.0,0,3,3
5185513,11,20,5,1635,36.0,3,4,4
...,...,...,...,...,...,...,...,...
2721992,6,21,7,630,-8.0,11,161,20
4210448,9,19,6,1922,-7.0,5,3,78
558299,2,6,5,1735,-5.0,4,51,14
2540125,6,10,3,905,-7.0,1,30,95


## Создание и обучение нейросети

Импортируем необходимые библиотеки

In [9]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Разделим датасет на обучающую и тестовую выборки

In [10]:
X = dataset.drop('DEPARTURE_DELAY', axis=1).values
y = dataset['DEPARTURE_DELAY'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Создадим модель

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

Обучим модель

In [12]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

epochs = 10

# tf.config.list_physical_devices('GPU')

history = model.fit(X_train, 
                    y_train, 
                    validation_split=0.2, 
                    epochs=epochs, 
                    batch_size=32)

Epoch 1/10
1970/1970 [==============================] - 4s 2ms/step - loss: 1445.4430 - accuracy: 0.0281 - val_loss: 1267.7236 - val_accuracy: 0.0284
Epoch 2/10
1970/1970 [==============================] - 4s 2ms/step - loss: 1439.1226 - accuracy: 0.0288 - val_loss: 1266.7211 - val_accuracy: 0.0296
Epoch 3/10
1970/1970 [==============================] - 4s 2ms/step - loss: 1437.3776 - accuracy: 0.0287 - val_loss: 1265.0542 - val_accuracy: 0.0283
Epoch 4/10
1970/1970 [==============================] - 3s 2ms/step - loss: 1436.2041 - accuracy: 0.0288 - val_loss: 1265.7400 - val_accuracy: 0.0286
Epoch 5/10
1970/1970 [==============================] - 4s 2ms/step - loss: 1434.6201 - accuracy: 0.0288 - val_loss: 1262.3759 - val_accuracy: 0.0284
Epoch 6/10
1970/1970 [==============================] - 4s 2ms/step - loss: 1433.5570 - accuracy: 0.0290 - val_loss: 1262.0162 - val_accuracy: 0.0279
Epoch 7/10
1970/1970 [==============================] - 4s 2ms/step - loss: 1432.1150 - accuracy: 0.

Сохраним полученные веса в файл

In [13]:
model.save('model/flight_delay_model.h5')